# Time-Spacial Exploration

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make charts from `repteis` database with focus on time and space exploration. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.m
    

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
# alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8-sig', low_memory=False)

<br>

<font size=5>**Color palette per Order**</font>

The image below served as inspiration (https://color.adobe.com/create/image)

<img src="./src/paleta_cores.jpeg" width='500px'>

Cores: 

- verde_escuro: #284021
- verde_claro: #88BF11
- amarelo: #D9CB0B
- laranja: #D99311
- laranja_escuro: #BF4417
- marrom-_laro: #BF8D7A

In [3]:
# importing customized color palettes
from src.MNViz_colors import *

In [4]:
# p.s.: Caudata is an error and should be removed
# cores_ordem = {
#     'Squamata': '#BF4417',
#     'Testudines': '#D9CB0B', 
#     'Crocodylia': '#284021'
# }

ordens = list(cores_ordem.keys())
cores = list(cores_ordem.values())

**Paleta de Cores - Família:**

- grupo 1: 1 cor  (verde escuro)
<ul>
    <li style="color:#142611"><b>centroide 1</b></li>
</ul>
- grupo 2: 8 cores
<ul>
    <li style="color:#85D907"><b>centroide 2</b></li>
</ul>

['#d7ff81', '#bafd62', '#9feb3f', '#85d907', '#6cc700', '#52b700', '#35a600', '#0b9700', '#008800']

<font color="#d7ff81"><b>cor 1</b> (ficou fora)</font>
<font color="#bafd62"><b>cor 2</b></font>
<font color="#9feb3f"><b>cor 3</b></font>
<font color="#85d907"><b>cor 4</b></font>
<font color="#6cc700"><b>cor 5</b></font>
<font color="#52b700"><b>cor 6</b></font>
<font color="#35a600"><b>cor 7</b></font>
<font color="#0b9700"><b>cor 8</b></font>
<font color="#008800"><b>cor 9</b></font>


- grupo 3: 2 cores
#888C03
<ul>
    <li style="color:#22401E"><b>centroide 3 (puxando para tons frios mais claros)</b></li>
</ul>

<font color="#99b6b2"><b>cor 1</b></font>
<font color="#81a58b"><b>cor 2</b></font>


- grupo 4: 1 cor  (amarelo)
<ul>
    <li style="color:#F2CB07"><b>centroide 4</b></li>
</ul>

- grupo 5: 10 cores
<ul>
    <li style="color:#cb97d4"><b>centroide 5 (puxando para o roxo)</b></li>
</ul>

['#f8dcf9', '#ebc5ed', '#ddafe2', '#ce9ad6', '#bf86cc', '#af73c2', '#a160b8', '#924fae', '#833fa4'] #803da1

<font color="#f8dcf9"><b>cor 1</b></font>
<font color="#ebc5ed"><b>cor 2</b></font>
<font color="#ddafe2"><b>cor 3</b></font>
<font color="#ce9ad6"><b>cor 4</b></font>
<font color="#bf86cc"><b>cor 5</b></font>
<font color="#af73c2"><b>cor 6</b></font>
<font color="#a160b8"><b>cor 7</b></font>
<font color="#924fae"><b>cor 8</b></font>
<font color="#833fa4"><b>cor 9</b></font>
<font color="#803da1"><b>cor 10</b></font>


- grupo 6: 12 cores
<ul>
    <li style="color:#91F2E9"><b>centroide 6</b></li>
</ul>

['#c9fff9', '#b3eff2', '#9cdcea', '#83c9e2', '#68b7da', '#4aa6d2', '#2096ca', '#0087c1', '#0079b7']

['#cee5d8', '#b3d2d1', '#9bbfc9', '#83adc2', '#6d9bba', '#568ab2', '#3e7baa', '#226ca2', '#005e98']


<font color="#c9fff9"><b>cor 1</b></font>
<font color="#b3eff2"><b>cor 2</b></font>
<font color="#9cdcea"><b>cor 3</b></font>
<font color="#83c9e2"><b>cor 4</b></font>
<font color="#68b7da"><b>cor 5</b></font>
<font color="#4aa6d2"><b>cor 6</b></font>
<font color="#2096ca"><b>cor 7</b></font>
<font color="#0087c1"><b>cor 8</b></font>
<font color="#0079b7"><b>cor 9</b></font>
<font color="#3e7baa"><b>cor 10</b></font>
<font color="#226ca2"><b>cor 11</b></font>
<font color="#005e98"><b>cor 12</b></font>


- grupo 7: 3 cores
<ul>
    <li style="color:#8C1A0F"><b>centroide 7 (puxando para o marrom)</b></li>
</ul>

['#fde5bf', '#efd09f', '#e1bb82', '#d3a767', '#c6934d', '#b98033', '#ac6f18', '#9e5e00', '#914e00']

<font color="#ac6f18"><b>cor 1</b></font>
<font color="#9e5e00"><b>cor 2</b></font>
<font color="#914e00"><b>cor 3</b></font>


- grupo 8: 13 cores
<ul>
    <li style="color:#D9430D"><b>centroide 8</b></li>
</ul>

['#ffce9f', '#ffb683', '#ff9f69', '#ff8851', '#f5723b', '#e75b25', '#d9430d', '#cb2800', '#bc0000']

['#ff8f68', '#ff7e56', '#ff6b40', '#ee5829', '#d9430d', '#c62f00', '#b41b00', '#a40300', '#930000']

<font color="#ffce9f"><b>cor 1</b></font>
<font color="#ffb683"><b>cor 2</b></font>
<font color="#ff9f69"><b>cor 3</b></font>
<font color="#ff8851"><b>cor 4</b></font>
<font color="#f5723b"><b>cor 5</b></font>
<font color="#e75b25"><b>cor 6</b></font>
<font color="#d9430d"><b>cor 7</b></font>
<font color="#cb2800"><b>cor 8</b></font>
<font color="#bc0000"><b>cor 9</b></font>
<font color="#c62f00"><b>cor 10</b></font>
<font color="#b41b00"><b>cor 11</b></font>
<font color="#a40300"><b>cor 12</b></font>
<font color="#930000"><b>cor 13</b></font>

<br>

**TOTAL: 50 cores**

In [5]:
# cores_familia = {
#     # known errors treatment
# #     '#n/d':'#000000',
# #     'nan':'#000000',
#     # grupo 1: Crocodylia
#     'Alligatoridae':'#142611',
#     # grupo 2: Testudines - Cryptodira
#     'Cheloniidae':'#bafd62',
#     'Chelydridae':'#9feb3f',
#     'Dermochelyidae':'#85d907',
#     'Emydidae':'#6cc700',
#     'Geoemydidae':'#52b700',
#     'Kinosternidae':'#35a600',
#     'Testudinidae':'#0b9700',
#     'Trionychidae':'#008800',
#     # grupo 3: Testudines - Pleurodira
#     'Chelidae':'#99b6b2' ,
#     'Podocnemididae':'#81a58b',
#     # grupo 4: Amphisbaenia - Amphisbaenia
#     'Amphisbaenidae':'#F2CB07',
#     # grupo 5: Sauria - Iguania
#     'Agamidae':'#f8dcf9',
#     'Chamaeleonidae':'#ebc5ed',
#     'Dactyloidae':'#ddafe2',
#     'Iguanidae':'#ce9ad6',
#     'Hoplocercidae':'#bf86cc',
#     'Leiosauridae':'#af73c2',
#     'Liolaemidae':'#a160b8', 
#     'Phrynosomatidae':'#924fae',
#     'Polychrotidae':'#833fa4',
#     'Tropiduridae':'#803da1',
#     # grupo 6: Sauria - Scleroglossa
#     'Anguidae':'#c9fff9',
#     'Gekkonidae':'#b3eff2',
#     'Gymnophthalmidae':'#9cdcea',
#     'Helodermatidae':'#83c9e2',
#     'Lacertidae':'#68b7da',
#     'Mabuyinae':'#4aa6d2',
#     'Phyllodactylidae':'#2096ca',
#     'Scincidae':'#0087c1',
#     'Sphaerodactylidae':'#0079b7',
#     'Teiidae':'#3e7baa',
#     'Varanidae':'#226ca2',
#     'Xantusiidae':'#005e98',
#     # grupo 7: Serpentes - Scolecophidia
#     'Anomalepididae':'#ac6f18',
#     'Leptotyphlopidae':'#9e5e00',
#     'Typhlopidae':'#914e00', 
#     # grupo 8: Alethinophidia
#     'Aniliidae':'#ffce9f',
#     'Boidae':'#ffb683',
#     'Colubridae':'#ff9f69',
#     'Dipsadidae':'#ff8851',
#     'Elapidae':'#f5723b',
#     'Homalopsidae':'#e75b25',
#     'Lamprophiidae':'#d9430d', 
#     'Loxocemidae':'#cb2800',
#     'Natricidae':'#bc0000',
#     'Pythonidae':'#c62f00',
#     'Tropidophiidae':'#b41b00',
#     'Viperidae':'#a40300',
#     'Xenopeltidae':'#930000'
# }

<br>

## Graph: Exploring time-spacial evolution of the database

In [6]:
# corrects a typo (Améica do Sul)
NewTable['continente'] = NewTable['continente'].apply(lambda x: 'América do Sul' if x=='Améica do Sul' else x)

In [7]:
# looking good...
NewTable['continente'].value_counts()

América do Sul      21927
América Central       135
América do Norte      103
Ásia                   54
#N/D                   25
África                 13
Europa                  9
Name: continente, dtype: int64

### Chart 1: Evolution per time and continent

In [8]:
# agrupando por Ano e Continente
teste = NewTable.groupby(['ano_coleta','continente', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

In [11]:
g1 = alt.Chart(teste, title='Evolução temporal da coleção por continente', height=300).mark_circle().encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('continente', title='Continente', 
             sort=alt.EncodingSortField('counts', op="count", order='descending')), 
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
#     color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),  # old palette per order
    color= alt.Color('familia',title= 'Família', 
                    legend= alt.Legend(columns=4, symbolLimit= 50),
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['continente','ano_coleta','familia','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_continente-FAMILIA.html')

# g1

<br>

### Chart 2: Evolution per time and country

In [12]:
# agrupando por Ano e Continente
teste1 = NewTable.groupby(['ano_coleta','pais', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

In [14]:
# retirei Brasil (será detalhado por Estados depois)
g1 = alt.Chart(teste1[teste1['pais'] != 'Brasil'], title='Animais coletados por ano em outros países',
              width=800).mark_circle().encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('pais', title='País', 
             sort=alt.EncodingSortField('counts', op="sum", order='descending')), 
    size=alt.Size('counts', scale= alt.Scale(range=[20,250])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
#     color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),  # old palette per order
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['pais','ano_coleta','familia','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_pais.html')

# g1

<br>

### Chart 3: Evolution per time and brazilian states

In [15]:
# agrupando por Ano e Continente
teste2 = NewTable[NewTable['pais'] == 'Brasil']
teste2 = teste2.groupby(['ano_coleta','estado_ou_provincia', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

#### creating column with brazilian regions

In [16]:
regioes = {
    'Rio de Janeiro':'SE',
    'São Paulo':'SE',
    'Espírito Santo': 'SE',
    'Pernambuco':'NE',
    'Santa Catarina':'S',
    'Amazonas':'N',
    'Goiás':'CO',
    'Roraima':'N',
    'Pará':'N',
    'Mato Grosso':'CO',
    'Acre': 'N',
    'Bahia': 'NE',
    'Minas Gerais': 'SE',
    'Mato Grosso do Sul': 'CO',
    'Paraná': 'S',
    'Rondônia': 'N',
    'Ceará': 'NE',
    'Maranhão': 'N',
    'Rio Grande do Sul': 'S',
    'Paraíba': 'NE',
    'Distrito Federal': 'CO',
    'Alagoas': 'NE',
    'Amapá':'N',
    'Piauí': 'NE',
    'Brasília': 'CO',
    'Tocantins': 'N',
    'Rio Grande do Norte': 'NE',
    'Sergipe': 'NE',
    'Minas Gerais/Goiás/Distrito Federal': 'CO',
    'Santa Catarina-Rio Grande do Sul': 'S'
}

# criando coluna com as regiões
teste2['regiao'] = teste2['estado_ou_provincia'].apply(lambda x: regioes[str(x)])

# coluna com estado, regiao
teste2['regiao_e_estado'] = teste2['estado_ou_provincia'] + ', ' + teste2['regiao']

# ordenando por região e soma das contagens
sorting = teste2.groupby(['regiao_e_estado', 'regiao']).sum()['counts'].reset_index(
                                                                ).rename(columns={'counts':'soma'})
sorting = sorting.sort_values(['regiao','soma'], ascending=False)['regiao_e_estado'].unique()

In [19]:
g1 = alt.Chart(teste2, title='Animais coletados por ano nos Estados Brasileiros').mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
)

# OBS: marca o background da região que eu quiser, mas distorce as cores
# rule = alt.Chart(teste2[teste2['regiao'].isin(['SE','NE','CO'])], 
#                  title='Animais coletados por ano nos Estados Brasileiros').mark_rule(
#                     opacity=0.1, size=19, color='#dedede', filled=False, fill='green').encode(
#     y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
#             sort= sorting, axis=alt.Axis(labels=True))
# )

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros.html')

# g1

<br>
<br>

<font color='red' size='5'> Separando por períodos </font>

In [20]:
# periodos: <1938, >=1938 e <= 1985, > 1985 
ano_corte1 = 1938
ano_corte2 = 1985

periodo1 = teste2[teste2['ano_coleta'] < 1938].copy()
periodo2 = teste2[(teste2['ano_coleta'] >= 1938) & (teste2['ano_coleta'] <= 1985)].copy()
periodo3 = teste2[teste2['ano_coleta'] > 1985].copy()

### periodo 1: até 1937

In [22]:
g1 = alt.Chart(periodo1, title='Animais coletados por ano nos Estados Brasileiros (até 1937)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros-ate1937.html')

# g1

### periodo 2: de 1938 a 1985

In [24]:
g1 = alt.Chart(periodo2, title='Animais coletados por ano nos Estados Brasileiros (de 1938 a 1985)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros-de1938a1985.html')

# g1

### periodo 3: após 1985

In [26]:
g1 = alt.Chart(periodo3, title='Animais coletados por ano nos Estados Brasileiros (após 1985)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Ano de Coleta'),
    y= alt.Y('regiao_e_estado', type="nominal",title='Estado, Região',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', scale= alt.Scale(range=[30,1200], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/evolucao_por_estados_brasileiros-apos1985.html')

# g1

<br>

## Lat Long

In [27]:
def regiao(est):
    if est not in regioes.keys():
        return np.NAN
    else:
        return regioes[est]
    
NewTable['regiao'] = NewTable['estado_ou_provincia'].apply(regiao)

In [28]:
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

world = alt.Chart(source).mark_geoshape(
    fill='white',
    stroke='gray'
).project('naturalEarth1')

# world

In [31]:
# dynamic version
slider = alt.binding_range(min=1900, max=2016, step=1)
select_year = alt.selection_single(name=' ', fields=['ano_coleta'],
                                   bind=slider, init={'ano_coleta': 1935})

# data = NewTable[(NewTable['long'] < 10) & (NewTable['long'] >-150)].copy()
data = NewTable.copy()

back = alt.Chart(data).mark_circle(opacity=0.7, color='lightgray').encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'ordem'])
)

front = alt.Chart(data).mark_circle().encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'ordem', 'familia'])
).add_selection(
    select_year
).transform_filter(
    select_year
)

equador = alt.Chart(data[data['lat'] == 0], width=800, height= 400).mark_rule(color='black').encode(
    y = alt.Y('lat:Q', axis=alt.Axis(grid=False))
)


g = alt.layer(world, back, front).properties(
    width= 800,
    height= 500
)  # tirei o equador

# ajustando tamanho de fonte
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/time-spacial/lat_long-worldwide.html')

# g

<br>

<font color='red' size='5'>Lat long (layout 2 estático)</font>

In [33]:
# https://nextjournal.com/sdanisch/cartographic-visualization

# seletores
select_order = alt.selection_multi(fields=['familia'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

tipos = NewTable['type_status'].unique()  # lista temporária! ajustar depois!

# AJUSTAR CORES

from vega_datasets import data

front = alt.Chart(NewTable).mark_point(filled=True).encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos),
                      legend= alt.Legend(columns=2)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','ordem','familia', 'type_status'])
).add_selection(select_order, select_type).transform_filter(select_order).transform_filter(select_type)

mapa = alt.layer(
     # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#f2f2f2'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(data.world_110m.url, 'countries')).mark_geoshape(
        fill='#white', stroke='black', strokeWidth=0.5
    ),
    front
).properties(
    width=800,
    height=500
)

mapa = mapa.project(
    type='naturalEarth1', scale=140
).configure_view(stroke=None)

mapa = mapa.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# mapa.save('./graphs/time-spacial/lat_long-worldwide-static-grid.html')

# mapa

In [35]:
# seletores
select_order = alt.selection_multi(fields=['familia'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

teste = alt.Chart(NewTable).mark_point(filled=True).encode(
    longitude = alt.X('long:Q', title='Longitude'),
    latitude = alt.Y('lat:Q', title='Latitude'),
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos), 
                     legend= alt.Legend(columns=2)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','ordem', 'familia', 'type_status'])
).project(type='naturalEarth1').add_selection(select_order, 
                            select_type).transform_filter(select_order).transform_filter(select_type)

temp = (world + teste).properties(width=800, height=500).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/time-spacial/lat_long-worldwide-static.html')
# temp

<br>

### Latin America

In [37]:
# seletores
select_order = alt.selection_multi(fields=['familia'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

tipos = NewTable['type_status'].unique()  # lista temporária! ajustar depois!

# mapa
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

latinAmerica = alt.Chart(source).mark_geoshape(
    fill='white',
    stroke='gray'
).project('naturalEarth1', translate=[680, 150], scale=450)

# pontos
teste = alt.Chart(NewTable).mark_point(filled=True).encode(
    longitude = alt.X('long:Q', title='Longitude'),
    latitude = alt.Y('lat:Q', title='Latitude'),
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos), 
                     legend= alt.Legend(columns=2)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','ordem', 'familia', 'type_status'])
).project(type='naturalEarth1', translate=[680, 150], scale=450).add_selection(select_order,
                                select_type).transform_filter(select_order).transform_filter(select_type)


# configurando grafico
temp = (latinAmerica + teste).properties(width=600, height=600).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/time-spacial/lat_long-latinAmerica-static.html')
# temp

**Fim!**